In [1]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.utils.prune as prune
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch.optim.lr_scheduler
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
from torchvision.models import alexnet
# from thop import profile
# from torch.utils.tensorboard import SummaryWriter

In [2]:
train_batch_size = 64
test_batch_size=100
num_classes = 100
learning_rate = 0.001
num_epochs = 100
lambda_l2 = 0.01
pruning_amount = 0.2
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
transform_train=transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [10]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=2)
valid_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                         shuffle=False, num_workers=2)

  0%|          | 0/170498071 [00:00<?, ?it/s]

In [3]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
class VGG(nn.Module):
  def __init__(self):
    super(VGG,self).__init__()
    self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
    self.bn1_1 = nn.BatchNorm2d(64)
    self.relu1_1 = nn.ReLU(inplace=True)
    self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
    self.bn1_2 = nn.BatchNorm2d(64)
    self.relu1_2 = nn.ReLU(inplace=True)
    self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
    self.bn2_1 = nn.BatchNorm2d(128)
    self.relu2_1 = nn.ReLU(inplace=True)
    self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
    self.bn2_2 = nn.BatchNorm2d(128)
    self.relu2_2 = nn.ReLU(inplace=True)
    self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
    self.bn3_1 = nn.BatchNorm2d(256)
    self.relu3_1 = nn.ReLU(inplace=True)
    self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
    self.bn3_2 = nn.BatchNorm2d(256)
    self.relu3_2 = nn.ReLU(inplace=True)
    self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
    self.bn3_3 = nn.BatchNorm2d(256)
    self.relu3_3 = nn.ReLU(inplace=True)
    self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
    self.bn4_1 = nn.BatchNorm2d(512)
    self.relu4_1 = nn.ReLU(inplace=True)
    self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.bn4_2 = nn.BatchNorm2d(512)
    self.relu4_2 = nn.ReLU(inplace=True)
    self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.bn4_3 = nn.BatchNorm2d(512)
    self.relu4_3 = nn.ReLU(inplace=True)
    self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.bn5_1 = nn.BatchNorm2d(512)
    self.relu5_1 = nn.ReLU(inplace=True)
    self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.bn5_2 = nn.BatchNorm2d(512)
    self.relu5_2 = nn.ReLU(inplace=True)
    self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.bn5_3 = nn.BatchNorm2d(512)
    self.relu5_3 = nn.ReLU(inplace=True)
    self.maxpool5 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.avgpool = nn.AvgPool2d(kernel_size=1,stride=1)
    self.classifier = nn.Sequential(
        nn.Linear(512 * 7 * 7, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(),
        nn.Linear(4096, 10)  # Adjust the output size for the number of classes in your task
    )
  def forward(self,x):
    x = self.conv1_1(x)
    x = self.bn1_1(x)
    x = self.relu1_1(x)
    x = self.conv1_2(x)
    x = self.bn1_2(x)
    x = self.relu1_2(x)
    x = self.maxpool1(x)

    x = self.conv2_1(x)
    x = self.bn2_1(x)
    x = self.relu2_1(x)
    x = self.conv2_2(x)
    x = self.bn2_2(x)
    x = self.relu2_2(x)
    x = self.maxpool2(x)

    x = self.conv3_1(x)
    x = self.bn3_1(x)
    x = self.relu3_1(x)
    x = self.conv3_2(x)
    x = self.bn3_2(x)
    x = self.relu3_2(x)
    x = self.conv3_3(x)
    x = self.bn3_3(x)
    x = self.relu3_3(x)
    x = self.maxpool3(x)

    x = self.conv4_1(x)
    x = self.bn4_1(x)
    x = self.relu4_1(x)
    x = self.conv4_2(x)
    x = self.bn4_2(x)
    x = self.relu4_2(x)
    x = self.conv4_3(x)
    x = self.bn4_3(x)
    x = self.relu4_3(x)
    x = self.maxpool4(x)

    x = self.conv5_1(x)
    x = self.bn5_1(x)
    x = self.relu5_1(x)
    x = self.conv5_2(x)
    x = self.bn5_2(x)
    x = self.relu5_2(x)
    x = self.conv5_3(x)
    x = self.bn5_3(x)
    x = self.relu5_3(x)
    x = self.maxpool5(x)

    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    x = self.classifier(x)
    return x

In [5]:
model = VGG().to(device)
# optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
# criterion = nn.CrossEntropyLoss()
# scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
# total_step = len(train_loader)
# loss_list = []
# acc_list = []
# writer = SummaryWriter()

In [ ]:
conv1_1=model.conv1_1
print(list(conv1_1.named_parameters()))

In [17]:
prune.random_unstructured(conv1_1,name="weight",amount=0.5)

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [ ]:
conv1_1.weight

In [ ]:
print(list(conv1_1.named_parameters()))

In [ ]:
conv1_1.weight_mask

In [22]:
prune.l1_unstructured(conv1_1,name="weight",amount=0.5)

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [25]:
conv1_1.weight

tensor([[[[-0.1486, -0.0000, -0.0000],
          [ 0.0000, -0.0000, -0.1625],
          [-0.0000, -0.0000, -0.1638]],

         [[-0.1615,  0.0000, -0.0000],
          [ 0.1769, -0.0000,  0.0000],
          [ 0.1423,  0.0000,  0.0000]],

         [[-0.0000,  0.0000,  0.0000],
          [ 0.1842,  0.0000, -0.0000],
          [-0.0000, -0.1821, -0.1635]]],


        [[[-0.0000,  0.0000, -0.0000],
          [-0.1611,  0.1052, -0.0000],
          [ 0.0000,  0.1434, -0.0000]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.1089,  0.0000]],

         [[-0.0000, -0.0000, -0.0000],
          [-0.0000, -0.1470, -0.1321],
          [ 0.0000,  0.0000,  0.1294]]],


        [[[ 0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.1312,  0.0000],
          [ 0.0000, -0.0000, -0.0000]],

         [[-0.0000,  0.0000, -0.1555],
          [ 0.0000, -0.1685,  0.0000],
          [-0.1667,  0.1911,  0.0000]],

         [[-0.0000, -0.0000, -0.1804],
     

In [26]:
conv1_1.weight_mask

tensor([[[[1., 0., 0.],
          [0., 0., 1.],
          [0., 0., 1.]],

         [[1., 0., 0.],
          [1., 0., 0.],
          [1., 0., 0.]],

         [[0., 0., 0.],
          [1., 0., 0.],
          [0., 1., 1.]]],


        [[[0., 0., 0.],
          [1., 1., 0.],
          [0., 1., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 1., 0.]],

         [[0., 0., 0.],
          [0., 1., 1.],
          [0., 0., 1.]]],


        [[[0., 0., 0.],
          [0., 1., 0.],
          [0., 0., 0.]],

         [[0., 0., 1.],
          [0., 1., 0.],
          [1., 1., 0.]],

         [[0., 0., 1.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        ...,


        [[[1., 1., 0.],
          [1., 0., 0.],
          [0., 1., 0.]],

         [[0., 0., 1.],
          [1., 0., 1.],
          [0., 0., 0.]],

         [[1., 0., 0.],
          [1., 1., 0.],
          [0., 1., 0.]]],


        [[[1., 0., 0.],
          [0., 0., 0.],
          [0., 1., 0.]],

         [[1., 